# Ejemplo con Scala RDDs  <span style="color: green">[SOLUCION]</span>

El típico _cuentapalabras_ , en Scala usando RDDs.

In [ ]:
// Which version of Scala?
scala.util.Properties.versionNumberString

## Carga de Spark

In [ ]:
// Load the Spark Jars into the Scala interpreter
def loadSpark(path: Option[String] = None ) = {
    // Directory where to look for jars to load
    val jardir = path.getOrElse("/opt/spark/current") + "/jars"
    // Get all jars in the directory
    val jarlist = java.nio.file.Files.list(java.nio.file.Paths.get(jardir)).toArray.map(_.toString);
    // Convert into a sequence of os.Path objects that can be loaded 
    val seq = jarlist.map( fname => ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(fname)) );
    // Load all jars into the interpreter
    interp.load.cp(seq);
}

loadSpark(sys.env.get("SPARK_HOME"));

In [ ]:
// Reduce a bit the logs from Spark to avoid too much information in the notebook
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org.apache").setLevel(Level.WARN)

In [ ]:
import org.apache.spark.sql._

val spark = SparkSession.builder().master("local[*]").getOrCreate()

## Carga de datos

La primera operación con el API RDD de Scala es obtener un `SparkContext`. En nuestro caso el kernel del _Notebook_ proporciona uno ya creado, en la variable `sc`

a través del `SparkContext` podemos cargar un fichero de disco

In [ ]:
val sc = spark.sparkContext
val textFile = sc.textFile("../../DATA/DonQuijote.txt.bz2")

## Procesado

In [ ]:
val words = textFile.flatMap( line => line.stripLineEnd.split(" ") )

In [ ]:
val initialcounts = words.map( word => (word, 1) )

In [ ]:
val counts = initialcounts.reduceByKey(_ + _)

recogemos los resultados

In [ ]:
for ( n <- counts.take( 10 ) ) println( n )

## Ordenación de resultados

Si queremos ordenar los resultados, podemos usar `sortByKey()`

In [ ]:
val sorted_counts = counts.sortByKey()

In [ ]:
for ( n <- sorted_counts.take( 10 ) ) println( n )

La clave, sin embargo, es la palabra, así que ese es el orden resultante. Si queremos ordenar por frecuencia de palabra (más frecuente primero), podemos intercambiar clave y valor en la tupla, y a continuación ordenar.

Para ello tenemos en cuenta que 
 * Cuando iteremos nuestro RDD lo que vamos a recibir por cada elemento es un objeto de la clase [`Tuple2`](http://www.scala-lang.org/api/2.11.x/index.html#scala.Tuple2), con el que hay que trabajar
 * Para controlar el sentido de la ordenación usamos los parámetos de llamada a [`sortByKey`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.OrderedRDDFunctions@sortByKey(ascending:Boolean,numPartitions:Int):org.apache.spark.rdd.RDD[(K,V)])

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>

<br/>
<div class="alert alert-block alert-info">Mirando la estructura y los métodos de <code>Tuple2</code>, podemos encontrar al menos 3 formas de hacerlo:</div>

In [ ]:
// Solución 1: creamos una nueva tupla con los valores intercambiados
val sorted_counts2a = counts.map( a => (a._2,a._1) ).sortByKey( ascending=false )

In [ ]:
// Solucion 2: usamos el método swap de la clase Tuple2
val sorted_counts2b = counts.map( a => a.swap ).sortByKey( false )

In [ ]:
// Solución 3: parámetro implícito
val sorted_counts2c = counts.map( _.swap ).sortByKey( false )

Veamos el resultado

In [ ]:
for ( n <- sorted_counts2c.take( 20 ) ) println( n )